In [ ]:
from kafka import KafkaConsumer, KafkaProducer
import json
import time

MAX_RETRIES = 2

consumer = KafkaConsumer(
    'lab6-topic',
    'lab6-topic-retry',
    bootstrap_servers='kafka_streaming_lab:9092',
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='lab6-processor',
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

producer = KafkaProducer(
    bootstrap_servers='kafka_streaming_lab:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

def process(msg):
    if msg['value'] == 'FAIL':
        raise Exception("Simulated failure")
    print(f"Processed: {msg}")

print("Retry processor running...")

for message in consumer:
    data = message.value
    try:
        process(data)
    except Exception as e:
        retries = data.get('retries', 0)
        if retries < MAX_RETRIES:
            data['retries'] = retries + 1
            producer.send('lab6-topic-retry', value=data)
            print(f"Retrying ({data['retries']}): {data}")
        else:
            producer.send('lab6-topic-dlq', value=data)
            print(f"Moved to DLQ: {data}")
    time.sleep(1)
